In [1]:
def CopyDynamicsData(lakeName):
  print("LakeName:", lakeName)
  
  # get last date from the loaded dataset
  LastSyncDate = GetLastSyncDate('opportunity', 'dynamics')
  
  print("Reading data since", LastSyncDate)  
  print("Load Opportunities data")
  PrepareDataBronzeWithoutInstance (
    lakeName = lakeName,
    entity = BronzeTable.Opportunities,
    entityName = 'tmpOpportunities',
    startDate = LastSyncDate
  )
  
  print("Writing data...")
  
  sourceData = spark.sql("""
    select
          o.Accountid
        , o.Statecode
        , coalesce(os.StateCodeName, 'BAD DATA') as StateName
        , o.ModifiedOn
        , o.CreatedOn
        , o.ActualCloseDate
        , o.StatusCode
        , coalesce(osc.StatusCodeName, 'BAD DATA') as StatusName
        , o.new_oppleadsource
        , o.OpportunityId
        , current_timestamp() as SilverModifiedUtc
    from tmpOpportunities o
      left join tmpOpportunityStateCode os on os.statecode = o.statecode
      left join tmpOpportunityStatusCode osc on osc.statuscode = o.statuscode
  """)  
  
  sourceData.registerTempTable("SourceData")
   
  spark.sql("""
    merge into delta.`wasbs://silver@"""+lakeName+""".blob.core.windows.net/dynamics/opportunity` as t
    using SourceData as s
      on t.OpportunityId = s.OpportunityId
    when matched then update set 
        AccountId = s.AccountId
      , StateCode = s.StateCode
      , Statename = s.Statename
      , ModifiedOn = s.ModifiedOn
      , CreatedOn = s.CreatedOn
      , ActualCloseDate = s.ActualCloseDate
      , StatusCode = s.StatusCode
      , Statusname = s.StatusName
      , New_OppLeadSource = s.New_OppLeadSource
      , SilverModifiedUtc = s.SilverModifiedUtc
    when not matched then 
      insert ( OpportunityId, AccountId, StateCode, Statename, ModifiedOn, CreatedOn, ActualCloseDate, StatusCode, StatusName, New_OppLeadSource, SilverModifiedUtc )
      values ( OpportunityId, AccountId, StateCode, Statename, ModifiedOn, CreatedOn, ActualCloseDate, StatusCode, StatusName, New_OppLeadSource, SilverModifiedUtc )
  """);

  print("Writing data completed.")
  
  LastSyncDate = sourceData.agg({"modifiedon": "max"}).collect()[0][0]
  print(LastSyncDate)

  if LastSyncDate is not None:
    UpdateLastSyncDate(LastSyncDate, 'opportunity', 'dynamics')